# Chapter 16

# Exercise 8

Embedded Reber grammars were used by Hochreiter and Schmidhuber in their paper about LSTMs. They are artificial grammars that produce strings such as “BPBTSXXVPSEPE.” Check out Jenny Orr’s nice introduction to this topic. Choose a particular embedded Reber grammar (such as the one represented on Jenny Orr’s page), then train an RNN to identify whether a string respects that grammar or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar, and 50% that don’t.

In [ ]:
from collections import defaultdict
from random import choice, random, sample
from tensorflow import keras

import tensorflow as tf

In [ ]:
from reber import *

## Reber

In [ ]:
reber_edges = ((0,1,'B'), (1,2,'T'), (1,3,'P'), (2,2,'S'), (2,4,'X'), (3,3,'T'), (3,5,'V'), (4,3,'X'), (4,6,'S'), (5,4,'P'), (5,6,'V'), (6,None,'E'))

In [ ]:
node_dict = dict_from_edges(reber_edges)

In [ ]:
node_dict

In [ ]:
sentence = generate_sentence(node_dict)

In [ ]:
sentence

In [ ]:
string_from_sentence(sentence)

In [ ]:
unique_letters(sentence)

In [ ]:
unique_letters(reber_edges)

In [ ]:
sentence_edge = sentence[3]

In [ ]:
sentence_edge

In [ ]:
corrupted_sentence_edge = corrupt_edge(sentence_edge, reber_edges)

In [ ]:
corrupted_sentence_edge

In [ ]:
corrupted_sentence = corrupt_sentence(sentence, reber_edges, 2)

In [ ]:
corrupted_sentence

## Embedder Reber Grammar

In [ ]:
embedded_reber_edges = ((0,1,'B'), (1,2,'T'), (1,3,'P'), (2,4,reber_edges), (3,5,reber_edges), (4,6, 'T'), (5,6,'P'), (6,None,'E'))

In [ ]:
embedded_reber_edges = flatten_embedded_edges(embedded_reber_edges)

In [ ]:
embedded_reber_edges

In [ ]:
node_dict = dict_from_edges(embedded_reber_edges)

In [ ]:
node_dict

In [ ]:
sentence = generate_sentence(node_dict)

In [ ]:
sentence

In [ ]:
string_from_sentence(sentence)

In [ ]:
corrupt_sentence(sentence, embedded_reber_edges, 3)

## Generate Training Data

We will write a generator function that produces a reber sentence. With equal probability, the sentence will be corrupted (label 0). If corrupted, the number of corruptions is randonmly determined.

In [ ]:
def generate_reber_training_sample(max_corruptions, edges, node_dict, allowed_chars):
    sentence = generate_sentence(node_dict)
    if random() < .5:
        num_corruptions = choice(range(1,max_corruptions+1))
        sentence = corrupt_sentence(sentence, edges, num_corruptions)
        label = 0
    else:
        label = 1
    s = string_from_sentence(sentence)
    x = string_to_ids(s, allowed_chars)
    x = tf.ragged.constant(x, dtype=tf.int8, ragged_rank=0)
    y = tf.constant(label, dtype=tf.int8)
    return (x, y)

In [ ]:
def training_data_generator(max_corruptions, edges, n=10000):
    node_dict = dict_from_edges(edges)
    allowed_chars = unique_letters(edges)
    for i in range(n):
        yield generate_reber_training_sample(max_corruptions, edges, node_dict, allowed_chars)

## Train a model

In [ ]:
max_corruptions = 3
embedding_size = 5
input_dim = len(unique_letters(embedded_reber_edges)) + 1

In [ ]:
data = tf.data.Dataset.from_generator(lambda: training_data_generator(max_corruptions, embedded_reber_edges),
                                     output_types=(tf.int8, tf.int8), output_shapes=(tf.TensorShape([None]), tf.TensorShape([])))

In [ ]:
data = data.padded_batch(32).prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_size, mask_zero=True),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate = 0.01)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model.fit(data, epochs=20)

Let's see how well an LSTM layer works

In [ ]:
model_lstm = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_size, mask_zero=True),
    keras.layers.LSTM(30),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_lstm.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history_lstm = model_lstm.fit(data, epochs=20)

Finally, let's try a SimpleRNN

In [ ]:
model_rnn = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_size, mask_zero=True),
    keras.layers.SimpleRNN(30, return_sequences=True),
    keras.layers.SimpleRNN(30, return_sequences=True),
    keras.layers.SimpleRNN(30),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_rnn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history_rnn = model_rnn.fit(data, epochs=20)

# Exercise 9
_Exercise: Train an Encoder–Decoder model that can convert a date string from one format to another (e.g., from "April 22, 2019" to "2019-04-22")._

First, we need a method to generate dates in different formats

In [ ]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa

## Character level seq-to-seq model

In [ ]:
from date_translation import *

In [ ]:
x,y = generate_training_dates(100)

In [ ]:
CHARS = list(set(''.join(x)))

In [ ]:
x[0]

In [ ]:
CHARS.index(x[0][0])

In [ ]:
CHARS

In [ ]:
preprocess_dates(x)
preprocess_dates(y)

In [ ]:
np.random.seed(42)

X_train, Y_train = generate_training_data(20000)
X_valid, Y_valid = generate_training_data(2000)
X_test,  Y_test  = generate_training_data(2000)

In [ ]:
X_train[0]

In [ ]:
embedding_size = 32
max_char_in = tf.math.reduce_max(X_train).numpy()
max_char_out = tf.math.reduce_max(Y_train).numpy()
input_length = X_train.shape[1]
output_length = Y_train.shape[1]

In [ ]:
max_char_in

In [ ]:
encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=max_char_in+1, output_dim=embedding_size, input_shape=[input_length]),
    keras.layers.LSTM(128)
])

In [ ]:
decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(max_char_out+1, activation='softmax')
])

In [ ]:
model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(output_length),
    decoder
])

In [ ]:
optimizer = keras.optimizers.Nadam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

In [ ]:
X_train

# Exercise 11
_Use one of the recent language models (e.g., GPT) to generate more convincing Shakespearean text._

In [2]:
from random import choice, seed
from tensorflow import keras
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

## Load data

In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

## Tokenization
GPT-2 uses byte-pair encoding

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [5]:
print(tokenizer('Good morning all!'))

{'input_ids': [10248, 3329, 477, 0], 'attention_mask': [1, 1, 1, 1]}


## Model

In [6]:
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Text generation - No Fine Tuning

In [7]:
shakespeare_lines = shakespeare_text.split('\n')

In [8]:
seed(142)
prompt = choice(shakespeare_lines)
print(prompt)

And time it is, when raging war is done,


In [9]:
encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='tf')
print(encoded_prompt)

tf.Tensor([[ 1870   640   340   318    11   618 30082  1175   318  1760    11]], shape=(1, 11), dtype=int32)


In [11]:
num_sentences = 5
max_num_tokens = 50

In [12]:
generated_sequences = model.generate(
input_ids=encoded_prompt,
max_length = max_num_tokens,
do_sample = True,
temperature=1.0,
top_k=0,
top_p=0.9,
repetition_penalty=1.0,
num_return_sequences=num_sentences)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [13]:
for sequence in generated_sequences:
    sentence = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(sentence)
    print("-" * 80)

And time it is, when raging war is done, it is all but safe.

Zuhdi, had you helped the rebels today, should you give them the information you've told them?

Yeah, he's caught up.
--------------------------------------------------------------------------------
And time it is, when raging war is done, your precious timbers will no longer hold life." He did not pause, but continued saying, "And this is like to Caesar's hurt shoulders: no matter what the order will be, I
--------------------------------------------------------------------------------
And time it is, when raging war is done, as thou hast done with this country; and when stardom are brought to the point of ruin, as thy midst has been broken in wars, thou hast brought thither for re-m
--------------------------------------------------------------------------------
And time it is, when raging war is done, are those responsible for maintaining peace?"(13)

He continued by saying "peace-loving nations must persist in support of

## Fine tuning

Training script for fine tuning language models in huggingface is published [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling)

In [11]:
train_size = len(shakespeare_lines) * 90 // 100

In [16]:
with open("train_shakespeare.txt", 'w') as f:
    f.write('\n'.join(shakespeare_lines[:train_size]))

In [17]:
with open("valid_shakespeare.txt", 'w') as f:
    f.write('\n'.join(shakespeare_lines[train_size:]))

In [ ]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "train_shakespeare.txt" \
--do_train \
--validation_file "valid_shakespeare.txt" \
--do_eval \
--num_train_epochs 5 \
--per_gpu_train_batch_size 1 \
--output_dir /gpt2_shakespeare/

In [17]:
model_finetunes = TFGPT2LMHeadModel.from_pretrained("gpt2_shakespeare/", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [18]:
generated_sequences = model_finetunes.generate(
input_ids=encoded_prompt,
max_length = max_num_tokens,
do_sample = True,
temperature=1.0,
top_k=0,
top_p=0.9,
repetition_penalty=1.0,
num_return_sequences=num_sentences)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [19]:
for sequence in generated_sequences:
    sentence = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(sentence)
    print("-" * 80)

And time it is, when raging war is done,--I am not too much persuaded of you:--I thought it well when we did fight.KING RICHARD II:In all my years of service, I have seen your duty straight
--------------------------------------------------------------------------------
And time it is, when raging war is done,Here we take our daily leave.Go forth and lament our Lord of Somerset,Whilst you hear the clamour,How often he cries and what doth he say;And how his jealous
--------------------------------------------------------------------------------
And time it is, when raging war is done, it fears much: And new things are doth grow stale.KING RICHARD III:My lords and masters,In this day many lights have been banished,But as lately of ages scarce
--------------------------------------------------------------------------------
And time it is, when raging war is done,For you are come with the weapon of thy breastAnd the plume of thy bosom.QUEEN MARGARET:So so is my body, and with the end of th